Load the rfc model for baseline 

In [39]:
import joblib
import os
print(os.getcwd())

rfc = joblib.load('./random_forest_model.pkl')
rfc

/Users/mehmettasdemir/Desktop/MLPatternClassification


RandomForestClassifier(random_state=10)

# Load the validation data

In [40]:
# the recordings are in separate npy files in the validation folder
import numpy as np

folder = './validation_data/'
filename = '2_Florian_Heizung_aus.npy' # change this to the file you want to load
val_data = np.load(folder + filename)
# shape is  (1, 175, 854) but it should be (854, 175) so we need to remove the first dimension
val_data = val_data[0]
val_data = val_data.T
val_data.shape

(854, 175)

Run the baseline model on the validation data and save the results in csv file

In [41]:
from helpers import convert_int_to_label
# some information about the csv result file:
#  four columns: first is the filename, second is the predicted command and third is the probability of the prediction and fourth is timestamp
#  the filename is the name of the file in the validation folder
#  the predicted command is the command that the model predicts for the file
#  the probability is the probability of the predicted command
#  the timestamp is the time of the prediction, which we can calculate by multiplying the index of the prediction by the duration of each recording which are 25 ms per frame

import pandas as pd

def run_baseline_model(rfc, val_data, filename):
    # get the predictions and probabilities
    predictions_int = rfc.predict(val_data)
    # convert the numerical predictions to the commands by using the function in helpers.py
    predictions = []
    for i in range(len(predictions_int)):
        predictions.append(convert_int_to_label(int(predictions_int[i])))
    
    probabilities = rfc.predict_proba(val_data)
    # get the commands
    commands = rfc.classes_
    # get the timestamps
    timestamps = np.arange(len(predictions)) * 0.025
    # create the dataframe
    df = pd.DataFrame({'filename': [filename] * len(predictions), 'predicted_command': predictions, 'probability': np.max(probabilities, axis=1), 'timestamp': timestamps})
    return df

df = run_baseline_model(rfc, val_data, filename)
df

,filename,predicted_command,probability,timestamp
0,2_Florian_Heizung_aus.npy,Spiegel,0.096944,0.000
1,2_Florian_Heizung_aus.npy,Spiegel,0.096944,0.025
2,2_Florian_Heizung_aus.npy,Spiegel,0.096944,0.050
3,2_Florian_Heizung_aus.npy,kann,0.080000,0.075
4,2_Florian_Heizung_aus.npy,offen,0.090000,0.100
...,...,...,...,...
849,2_Florian_Heizung_aus.npy,other,0.220000,21.225
850,2_Florian_Heizung_aus.npy,other,0.200000,21.250
851,2_Florian_Heizung_aus.npy,other,0.230000,21.275
852,2_Florian_Heizung_aus.npy,other,0.230000,21.300
